In [1]:
#import necessary libraries
import pandas as pd
import nltk
import sys # needed this for certain print options during debugging
import numpy as np #lmao why did we not have this before
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soumyankar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#load the dataset
pd.set_option('display.max_colwidth', None)
data = pd.read_csv('final_dataset.csv')
data.head(2)

,review_id,title,tags,year,user_review,user_suggestion,url
0,1,Spooky's Jump Scare Mansion,"['Action', 'Adventure', 'Free to Play', 'Indie']",2016.0,"I'm scared and hearing creepy voices. So I'll pause for a moment and write a review while I wait for my heart beat to return to atleast somewhat calmer times. This game is adorable and creepy like my happy tree friends but with the graphics sceme of my childhood (but more bubble and 'clean'). Hello 1990's.What charactes there are (that isnot trying to kill me) were likable and a bit odd. I did do a few noob things though, such as:Oh look a class room full of ghosts from dead children, lets shine my flashlight on them and stand there staring at them..Or, hmm creepy music, I'll turn around and see if I can see what's chasing me.Never before in a game have I been this afraid of finding a locked door.",1,https://www.youtube.com/watch?v=48NzbFQebxg
1,2,Spooky's Jump Scare Mansion,"['Action', 'Adventure', 'Free to Play', 'Indie']",2016.0,"Best game, more better than Sam Pepper's YouTube account. 10/10What you'll need to play:A computerSome extra pants.Pros:Scary as hell.Fun.Adventure.Spooky.Did I forgot to mention that its scary as hell?You'll get more pants/briefs in your wardrobe.Time consuming if you're bored.Cons:Buying pants/briefs. You haven't downloaded it yet.",1,https://www.youtube.com/watch?v=48NzbFQebxg


In [3]:
#dataset facts
data.shape

(17494, 7)

In [4]:
#dataset facts
data['user_suggestion'].value_counts()

1    9968
0    7526
Name: user_suggestion, dtype: int64

In [5]:
#dataset facts
data['title'].nunique()

44

In [6]:
# Adding tags to the games here
data['title'].unique()

array(["Spooky's Jump Scare Mansion", 'Sakura Clicker', 'WARMODE',
       'Fractured Space', 'Dota 2', 'Path of Exile', 'War Thunder',
       'Ring of Elysium', 'Team Fortress 2', 'Yu-Gi-Oh! Duel Links',
       'SMITE®', 'Brawlhalla', 'World of Tanks Blitz',
       'DCS World Steam Edition', 'Heroes & Generals',
       'The Elder Scrolls®: Legends™', 'Trove', 'Neverwinter',
       'Realm Royale', 'PlanetSide 2', 'Realm of the Mad God', 'Elsword',
       'theHunter Classic', 'Eternal Card Game', 'Black Squad',
       'Freestyle 2: Street Basketball', 'Bless Online',
       'RaceRoom Racing Experience', 'Fallout Shelter',
       'Tactical Monsters Rumble Arena', 'Creativerse', 'Dreadnought',
       'Infestation: The New Z', 'Shop Heroes', 'Robocraft',
       'Business Tour - Board Game with Online Multiplayer',
       'Crusaders of the Lost Idols', 'AdventureQuest 3D',
       'Realm Grinder', 'World of Guns: Gun Disassembly',
       'Bloons TD Battles', 'Cuisine Royale', 'School of Drago

In [7]:
# Data processing for removing stop words, lemmatization and stemmming, and english-only.
# Importing all relevant libraries and functions here
import spacy #load spacy
from sklearn.pipeline import Pipeline # pipelining pre-processing
from spacy.lang.en import English
nlp = spacy.load("en_core_web_sm")
tokenizer = nlp.tokenizer

2023-02-04 14:37:56.655320: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
#tokenization
def create_tokenized_dataset(data):
    tokens_list = []

    for j in data['user_review']:
#         tokens_list.append(word_tokenize(j))
        spacy_tokens = [token.orth_ for token in tokenizer(j)]
        tokens_list.append(spacy_tokens)
    data['Tokens'] = tokens_list   
    return data

df = create_tokenized_dataset(data)
df.head(1)

,review_id,title,tags,year,user_review,user_suggestion,url,Tokens
0,1,Spooky's Jump Scare Mansion,"['Action', 'Adventure', 'Free to Play', 'Indie']",2016.0,"I'm scared and hearing creepy voices. So I'll pause for a moment and write a review while I wait for my heart beat to return to atleast somewhat calmer times. This game is adorable and creepy like my happy tree friends but with the graphics sceme of my childhood (but more bubble and 'clean'). Hello 1990's.What charactes there are (that isnot trying to kill me) were likable and a bit odd. I did do a few noob things though, such as:Oh look a class room full of ghosts from dead children, lets shine my flashlight on them and stand there staring at them..Or, hmm creepy music, I'll turn around and see if I can see what's chasing me.Never before in a game have I been this afraid of finding a locked door.",1,https://www.youtube.com/watch?v=48NzbFQebxg,"[I, 'm, scared, and, hearing, creepy, voices, ., , So, I, 'll, pause, for, a, moment, and, write, a, review, while, I, wait, for, my, heart, beat, to, return, to, atleast, somewhat, calmer, times, ., , This, game, is, adorable, and, creepy, like, my, happy, tree, friends, but, with, the, graphics, sceme, of, my, childhood, (, but, more, bubble, and, ', clean, ', ), ., , Hello, 1990's, ., What, charactes, there, are, (, that, isnot, trying, to, kill, me, ), were, likable, and, a, bit, odd, ., , I, did, do, a, few, noob, things, though, ,, such, as, ...]"


In [9]:
#define lemmatizer
lemmatizer = WordNetLemmatizer()

In [10]:
#define stopwords
stopWords = set(stopwords.words('english'))
custom_stop = ["'m", "'ll", "'s", "'re", "n't", "'ve"]
stop_words = set(list(stopWords) + custom_stop)

In [11]:
#define punctuation
custom_punc = ["``", "''", "--", "...","♥"]
punctuation = set(list(string.punctuation) + custom_punc)

In [12]:
# nltk for removing english words
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     /Users/soumyankar/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [13]:
#data preprocessing: define functions

# created a function to deal with that pesky steam heart emoticon
# print(ord('♥'))
def remove_steam_heart(tokens):
    for token in tokens:
        if len(token) == 1:
            if(ord(token) == 9829): # 9829 is the unicode for the heart emoticon
                tokens.remove(token)
    return tokens

#lower case
def lower_case(tokens):
    tokens = [token.lower() for token in tokens]
    return tokens

#remove stopwords
def remove_stopwords(tokens):
#     tokens = [token for token in tokens if token not in stop_words]
# Using the spacy stop words set instead of the english stop words
    tokens = [token for token in tokens if token not in nlp.Defaults.stop_words]
    return tokens

#remove digits
def remove_digits(tokens):
    for token in tokens:
        if token.isdigit():
            tokens.remove(token)
    return tokens

#remove punctuation
def remove_punctuation(tokens):
    tokens = [token for token in tokens if token not in punctuation]
    return tokens

#lemmatization
def lemmatize_words(tokens):
    for token in range(len(tokens)):
        tokens[token] = lemmatizer.lemmatize(tokens[token])
    return tokens

# Using nltk to word corpus to remove non-english words
# this method seemed cheaper and easier than spacy? maybe lol
def remove_non_english(tokens):
    tokens = [token for token in tokens if token in words]
    return tokens

In [14]:
#data preprocessing function
def data_preprocess(tokens):
    new_tokens = lower_case(tokens)
    new_tokens = remove_steam_heart(new_tokens)
    new_tokens = remove_non_english(new_tokens)
    new_tokens = remove_stopwords(new_tokens)
    new_tokens = remove_digits(new_tokens)
    new_tokens = remove_punctuation(new_tokens)
    new_tokens = lemmatize_words(new_tokens)
    return new_tokens
    
list_of_process_tokens = []
for i in df['Tokens']:
    list_of_process_tokens.append(data_preprocess(i))

In [15]:
df['ProcessTokens'] = list_of_process_tokens    

,review_id,title,tags,year,user_review,user_suggestion,url,Tokens,ProcessTokens
7816,10826,Trove,"['Action', 'Adventure', 'Casual', 'Free to Play', 'Massively Multiplayer', 'RPG']",2016.0,"One of my favorite steam games to play as of right now. This game brings me back to when Lego Universe existed, and it has a very similar feeling to it. Lots of classes, fair play, some minor tweaking needed here and there, but overall, great game.",1,https://www.youtube.com/watch?v=ns1i_2SzeGE,"[One, of, my, favorite, steam, games, to, play, as, of, right, now, ., This, game, brings, me, back, to, when, Lego, Universe, existed, ,, and, it, has, a, very, similar, feeling, to, it, ., Lots, of, classes, ,, fair, play, ,, some, minor, tweaking, needed, here, and, there, ,, but, overall, ,, great, game, .]","[favorite, steam, play, right, game, universe, similar, feeling, lot, class, fair, play, minor, overall, great, game]"
1529,1530,Fractured Space,"['Action', 'Indie', 'Early Access']",2016.0,"Early Access ReviewI play this game infrequently, and I enjoy it very much whenever I do play it. This is one of the best free to play games I have ever played, and if you haven't played it yet, you need to.",1,https://www.youtube.com/watch?v=MgKVV4pZBWo,"[Early, Access, ReviewI, play, this, game, infrequently, ,, and, I, enjoy, it, very, much, whenever, I, do, play, it, ., This, is, one, of, the, best, free, to, play, games, I, have, ever, played, ,, and, if, you, have, n't, played, it, yet, ,, you, need, to, .]","[early, access, play, game, infrequently, enjoy, play, best, free, play, need]"
14970,22081,Robocraft,"['Action', 'Free to Play', 'Indie', 'Massively Multiplayer', 'RPG', 'Simulation']",2016.0,"Early Access ReviewThis used to be my favirote game. I was excited for the meta, and it was great at first then came update after update, screw up after screw up and I couldn\'t figure out anything. The level system, the blocks, the guns, everything fundamentally changed and it stripped away everything even remotely enjoyable about the game. This game has great potential, but the devs preetty much ran it into the ground for money. I would play this game, PAY for it if it brought back everything to the first stage of the meta, with plenty of customization, a good level system, good gamemodes, and room to grow. Long story short, the devs had something good, something great and ruined it. I\'m sorry to say that I would NOT reccomend this game.",0,https://www.youtube.com/watch?v=JGr1Em2Ip-c,"[Early, Access, ReviewThis, used, to, be, my, favirote, game, ., I, was, excited, for, the, meta, ,, and, it, was, great, at, first, then, came, update, after, update, ,, screw, up, after, screw, up, and, I, couldn\'t, figure, out, anything, ., The, level, system, ,, the, blocks, ,, the, guns, ,, everything, fundamentally, changed, and, it, stripped, away, everything, even, remotely, enjoyable, about, the, game, ., This, game, has, great, potential, ,, but, the, devs, preetty, much, ran, it, into, the, ground, for, money, ., I, would, play, this, game, ,, PAY, for, it, if, it, brought, back, everything, to, ...]","[early, access, game, excited, meta, great, came, update, update, screw, screw, figure, level, system, fundamentally, stripped, away, remotely, enjoyable, game, game, great, potential, ran, ground, money, play, game, pay, brought, stage, meta, plenty, good, level, system, good, room, grow, long, story, short, good, great, ruined, sorry, game]"
4165,5561,Yu-Gi-Oh! Duel Links,"['Free to Play', 'Strategy']",2018.0,"This game is very beginner friendly and easy to pick up. I had never played yugioh before and this game was my first experience playing it. You start off with a decent deck (it isn't the best but for learning purposes, its pretty good) with a well known monster of your choice, Blue-Eyes White Dragon or Dark Magician. The gacha mechanic for gettting new cards is great and guarentees rare cards if you're patient enough to use enough gems on it. There is 

In [ ]:
# making the word soup column
for index,row in df.iterrows():
    current_row_soup = []
    current_row_tags = list(map(str.lower,row['tags']))
    if len(row['ProcessTokens']):
        current_row_soup = list(row['ProcessTokens'] + current_row_tags)
    else:
        current_row_soup = current_row_tags
    current_row_soup = ' '.join(map(str, current_row_soup))
    df.loc[index, 'soup'] = current_row_soup

In [ ]:
#Converting the datatype of processtokens fromm list to string, so that our vectorization works right.
df['ProcessTokens'] = df['ProcessTokens'].astype("string")

In [ ]:
# making a new column for word soup
if 'soup' not in df.columns:
    df.insert(6,'soup', "")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import chain

def make_recommendation(df=df):
    new_row = df.iloc[-1,:].copy() #creating a copy of the last row of the 
    #dataset, which we will use to input the user's input

    #grabbing the new wordsoup from the user
    searchTerms = getSearchTerms()
    new_row['soup'] = searchTerms
    new_row['title'] = 'UserInput'

    #adding the new row to the dataset
    df = df.append(new_row)
#     df.iloc[-1] = searchTerms #adding the input to our new row

    #Vectorizing the entire matrix as described above!
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(df['soup'])

    #running pairwise cosine similarity 
    cosine_sim2 = cosine_similarity(count_matrix, count_matrix) #getting a similarity matrix

    #sorting cosine similarities by highest to lowest
    sim_scores = list(enumerate(cosine_sim2[-1,:]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #matching the similarities to the movie titles and ids
    #   ranked_titles = []
    #   for i in range(1, 11):
    #     indx = sim_scores[i][0]
    #     ranked_titles.append([df['title'].iloc[indx], df['soup'].iloc[indx]])
    i = 10
    inc = 1 # we start from the 1-th row because, the 0th row is the input itself. dont want that.
    
    ranked_titles = []
    while i>=0:
        indx = sim_scores[inc][0]
        inc = inc + 1
        current_title = df['title'].iloc[indx]
        if(current_title == 'UserInput'):
            continue
        if(current_title not in chain(*ranked_titles)):
            ranked_titles.append([df['title'].iloc[indx], sim_scores[inc-1][1]])
            i = i - 1

    return ranked_titles